In [1]:
# importing prerequisites
import sys
import requests
import cv2
import random
import tarfile
import json
import numpy as np
import pdf2image
from os import path
from PIL import Image
from PIL import ImageFont, ImageDraw
from glob import glob
from matplotlib import pyplot as plt
from pdf2image import convert_from_path
from pathlib import Path
from PyPDF2 import PdfFileReader
from IPython.core.display import display, HTML
import pdb
import simplejson as json
import glob
import os
import copy

%matplotlib inline

In [2]:
# Verifying the file was extracted properly
data_path = "/media/ananta/KGP/FinsightAI/fintabnet"
path.exists(data_path)

True

In [3]:
# Parse the JSON file and read all the images and labels
with open('/media/ananta/KGP/FinsightAI/fintabnet/FinTabNet_1.0.0_table_val.jsonl', 'r') as fp:
    images = {}
    for line in fp:
        sample = json.loads(line)
        # Index images
        if sample['filename'] in images:
            annotations = images[sample['filename']]["annotations"]
#             html = images[sample['filename']]["html"]
        else:
            annotations = []
#             html = ""
        for t, token in enumerate(sample["html"]["cells"]):
            if "bbox" in token:
                annotations.append({"category_id":2, "bbox": token["bbox"]})
#         #Build html table
#         cnt = 0
#         for t, token in enumerate(sample["html"]["structure"]["tokens"]):
#             html += token
#             if token=="<td>":
#                 html += "".join(sample["html"]["cells"][cnt]["tokens"])
#                 cnt += 1
        annotations.append({"category_id": 1, "bbox": sample["bbox"]})
        images[sample['filename']] = {'filepath': '/media/ananta/KGP/FinsightAI/fintabnet/pdf/' + sample["filename"], 
#                                       'html': html, 
                                      'annotations': annotations}
        

In [13]:
# Function to viz the annotation
def markup(i,a,image, annotations,pdf_width, pdf_height,dic):
    ''' Draws the segmentation, bounding box, and label of each annotation
    '''
    img_width,img_height = image.size
#     print("btbtbt",img_height,img_width)
    filled_table = np.zeros_like(image)
    filled_cells = np.zeros_like(image)
    
    draw = ImageDraw.Draw(image, 'RGBA')
    for annotation in annotations:
        # Draw bbox
#         orig_annotation = copy.copy(annotation['bbox'])
# #         annotation['bbox'][3] = pdf_height-orig_annotation[1]
# #         annotation['bbox'][1] = pdf_height-orig_annotation[3]
#         draw.rectangle(
#             (annotation['bbox'][0]*img_width/pdf_width,
#              (float(pdf_height)-annotation['bbox'][3])*img_height/pdf_height,
#              (annotation['bbox'][2])*img_width/pdf_width,
#              (float(pdf_height)-annotation['bbox'][1])*img_height/pdf_height),
#             outline=colors[annotation['category_id'] - 1] + (255,),
#             width=2
#         )
        
        xmin=round((annotation['bbox'][0])*img_width/pdf_width)
        ymin=round((pdf_height-annotation['bbox'][3])*img_height/pdf_height)
        xmax=round((annotation['bbox'][2])*img_width/pdf_width)
        ymax=round((pdf_height-annotation['bbox'][1])*img_height/pdf_height)
        
        if annotation['category_id']==1:
            a+=1
            dic['annotations'].append({"id":len(dic['annotations'])+1,
                                       "iscrowd":0,
                                       "image_id":i+1,
                                       "category_id":1,
                                       "segmentation":[
                                           [
                                               xmin,ymin,
                                               xmax,ymin,
                                               xmax,ymax,
                                               xmin,ymax
                                           ]
                                       ],
                                       "bbox":[
                                           xmin,
                                           ymin,
                                           xmax-xmin,
                                           ymax-ymin,
                                       ],
                                       "area":img_width*img_height})
                

    return np.array(filled_table),np.array(filled_cells)

In [ ]:
import matplotlib
matplotlib.rcParams['figure.dpi'] = 505
dic = {"images":[],"annotations":[],"categories":[{"id":1,"name":"Table"}]}
a=0
for i, (filename, image) in enumerate(images.items()):
    
#     if i==10:
#         break
    filenaam = filename.split('.')[0].replace('/','_')
    pdf_page = PdfFileReader(open(image["filepath"], 'rb')).getPage(0)
    pdf_shape = pdf_page.mediaBox
    pdf_height = pdf_shape[3]-pdf_shape[1]
    pdf_width = pdf_shape[2]-pdf_shape[0]
    converted_images = convert_from_path(image["filepath"],size=(pdf_width, pdf_height))
    img = converted_images[0].copy()
    dic['images'].append({"id":i+1,"width":pdf_width,"height":pdf_height,"file_name":filenaam+'.jpg'})
    
    try:
        table_mask,cell_mask = markup(i,a,img, image['annotations'],pdf_width, pdf_height,dic)
        
    except:
        continue
    table_mask,cell_mask = Image.fromarray(table_mask),Image.fromarray(cell_mask)
    img = converted_images[0]
    img = img.resize((512,512))
    table_mask = table_mask.resize((512,512))
#     print(img.size)
#     img.save(f'Fintab_test_data/images/{filenaam}.jpg')
#     table_mask.save(f'Fintab_test_data/table_mask/{filenaam}.jpg')
#     cell_mask.save(f'Fintab_highres/cell_mask/{filenaam}.jpg')
    print(i)
    print(filenaam)
    

#     plt.figure()
#     plt.subplot(1,3,1)
#     plt.imshow(img)    
#     plt.title("Page # {}".format(i))
#     plt.subplot(1,3,2)
#     plt.imshow(table_mask)
#     plt.title("Table Mask # {}".format(i))
#     plt.subplot(1,3,3)
#     plt.imshow(cell_mask)
#     plt.title("Cell Mask # {}".format(i))
#     plt.show()


In [15]:
with open('Fintab_val_annot.json', 'w') as f:
    json.dump(dic, f)

In [12]:
dic

{'images': [{'id': 1,
   'width': 612,
   'height': 792,
   'file_name': 'FDX_2003_page_67.jpg'},
  {'id': 2, 'width': 612, 'height': 792, 'file_name': 'FDX_2017_page_64.jpg'},
  {'id': 3, 'width': 612, 'height': 792, 'file_name': 'FDX_2018_page_60.jpg'},
  {'id': 4, 'width': 612, 'height': 792, 'file_name': 'FDX_2005_page_83.jpg'},
  {'id': 5, 'width': 612, 'height': 792, 'file_name': 'FDX_2006_page_89.jpg'},
  {'id': 6, 'width': 612, 'height': 792, 'file_name': 'FDX_2018_page_62.jpg'},
  {'id': 7, 'width': 612, 'height': 792, 'file_name': 'FDX_2013_page_30.jpg'},
  {'id': 8, 'width': 612, 'height': 792, 'file_name': 'FDX_2015_page_35.jpg'},
  {'id': 9, 'width': 612, 'height': 792, 'file_name': 'FDX_2017_page_65.jpg'},
  {'id': 10,
   'width': 612,
   'height': 792,
   'file_name': 'FDX_2018_page_27.jpg'}],
 'annotations': [{'id': 1,
   'iscrowd': 0,
   'image_id': 1,
   'category_id': 1,
   'segmentation': [[66, 176, 311, 176, 311, 304, 66, 304]],
   'bbox': [66, 176, 245, 128],
   '